In [1]:
#Chequeo version XGBOost

import xgboost as xgb, subprocess, sys
print("xgboost:", xgb.__version__)
try:
    print(subprocess.check_output(["nvidia-smi"]).decode())
except Exception as e:
    print("nvidia-smi no disponible:", e, file=sys.stderr)


xgboost: 2.0.3
Fri Sep 12 18:37:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------------

In [2]:
import pandas as pd # Para cargar los datos y hacer OHE.
import numpy as np  # Para lidiar con NaNs.
import time
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import confusion_matrix



random_state = 42
np.random.seed(random_state)

In [3]:
train_data = pd.read_csv("/kaggle/input/datasetspotify/train_data.txt", sep="\t", low_memory=False, on_bad_lines='skip')
test_data = pd.read_csv("/kaggle/input/datasetspotify/test_data.txt", sep="\t", low_memory=False, on_bad_lines='skip')
print(train_data.shape)
print(test_data.shape)

(911344, 22)
(51570, 21)


In [4]:
# - Convertimos las columnas de fechas a tipo datetime (con zona horaria).
# - Crea nuevas columnas a partir de la fecha/hora:
#    * hour → hora del día (0–23)
#    * dayofweek → día de la semana (0=lunes … 6=domingo)
#    * is_weekend → marca si es sábado o domingo
#    * gap_offline_minutes → minutos entre ts y offline_timestamp (o -1 si no hay)
#    * had_offline → marca si existió un offline_timestamp
#
# - Estos datos reflejan cómo usan la app según hora o día.
# - Capturan patrones de comportamiento que ayudan al modelo a separar


# Asegurate que ts y offline_timestamp sean datetime con tz
train_data["ts"] = pd.to_datetime(train_data["ts"], utc=True, errors="coerce")
test_data["ts"]  = pd.to_datetime(test_data["ts"],  utc=True, errors="coerce")
train_data["offline_timestamp"] = pd.to_datetime(train_data["offline_timestamp"], utc=True, errors="coerce")
test_data["offline_timestamp"]  = pd.to_datetime(test_data["offline_timestamp"],  utc=True, errors="coerce")

# Derivadas de ts (baratas y rinden)
def add_time_feats(df):
    df["hour"] = df["ts"].dt.hour.astype("int16")
    df["dayofweek"] = df["ts"].dt.dayofweek.astype("int8")
    df["is_weekend"] = df["dayofweek"].isin([5,6]).astype("int8")
    return df

train_data = add_time_feats(train_data)
test_data  = add_time_feats(test_data)

# Gap con offline_timestamp (si hay)
def add_offline_gap(df):
    gap = (df["ts"] - df["offline_timestamp"]).dt.total_seconds() / 60.0
    df["gap_offline_minutes"] = gap.fillna(-1).astype("float32")  # -1 = no offline
    df["had_offline"] = df["offline_timestamp"].notna().astype("int8")
    return df

train_data = add_offline_gap(train_data)
test_data  = add_offline_gap(test_data)


In [5]:

# Normalizamos columna "platform"
# --------------------------------
# - Reduce los textos largos y variados de "platform" a grupos simples:
#  porque existen diferentes versiones de android, ios, windows, mac, linux, web, unknown.
# - Ejemplo: "Windows 7, 10, 13, etc (cualquier windows) → "windows"
#
# - Evita que el modelo cree miles de categorías distintas para lo mismo.
# - Mantiene solo la info importante: el tipo de dispositivo/sistema usado.
# - Mejora memoria y hace que el modelo aprenda más rápido y con menos ruido.

import re

def simplify_platform(s):
    if not isinstance(s, str): return "unknown"
    low = s.lower()
    if "android" in low: return "android"
    if "ios" in low or "iphone" in low or "ipad" in low: return "ios"
    if "windows" in low: return "windows"
    if "mac" in low or "os x" in low or "macos" in low: return "mac"
    if "linux" in low or "ubuntu" in low: return "linux"
    if "web" in low or "browser" in low: return "web"
    return re.sub(r"\s+", "_", low[:20])

train_data["platform_simpl"] = train_data["platform"].map(simplify_platform)
test_data["platform_simpl"]  = test_data["platform"].map(simplify_platform)


In [6]:
# Creo la variable objetivo
#--------------------------------
# - Definimos la columna "y" como target:
#     * y = 1 si reason_end == "fwdbtn"  (el usuario saltó la canción)
#     * y = 0 en los demás casos
# - Convierte a int8 para ahorrar memoria.
#
# - Necesitamos una variable binaria (0/1) para entrenar
#   el modelo de clasificación con XGBoost.

# y = (reason_end == 'fwdbtn')
train_data["y"] = (train_data["reason_end"] == "fwdbtn").astype("int8")


In [7]:
# Frequency Encoding para columnas grandes
# --------------------------------
# - Reemplazamos valores categóricos muy numerosos (ej: username, ip_addr, track_name)
#   por la cantidad de veces que aparecen en el dataset (su frecuencia).
# - Genera nuevas columnas: fe_username, fe_ip_addr, etc.
#
# Ejemplo:
#   si un username aparece 500 veces → fe_username = 500
#   si un track aparece solo 3 veces → fe_track_name = 3

def freq_encode(train_col, test_col):
    vc = train_col.value_counts()
    mapper = vc.astype("int64")
    return train_col.map(mapper).fillna(0).astype("int32"), test_col.map(mapper).fillna(0).astype("int32")

# Elegí columnas grandes
big_cols = [
    "username", "ip_addr",
    "master_metadata_track_name",
    "master_metadata_album_artist_name",
    "master_metadata_album_album_name",
    "spotify_track_uri"
]

for col in big_cols:
    tr_fe, te_fe = freq_encode(train_data[col].astype("object"), test_data[col].astype("object"))
    train_data[f"fe_{col}"] = tr_fe
    test_data[f"fe_{col}"]  = te_fe


In [8]:

#One-Hot Encoding (OHE) para platform y conn_country
# --------------------------------
# - Tomamos solo los valores más frecuentes para reducir cardinalidad:
#     * platform_simpl → top 10
#     * conn_country   → top 40
# - Los valores fuera del top se agrupan en "other".
# - Convierte cada categoría en una columna binaria con get_dummies.
# - Alinea las columnas entre train y test (rellena faltantes con 0).
#
# Ejemplo:
#   platform_simpl = "windows" → platform_simpl_windows = 1
#   platform_simpl = "other"   → todas las demás columnas = 0, salvo "other" = 1


def cap_to_topN(train_series, test_series, topN=30):
    top = train_series.value_counts().nlargest(topN).index
    tr = train_series.where(train_series.isin(top), "other")
    te = test_series.where(test_series.isin(top), "other")
    return tr, te

train_plat, test_plat = cap_to_topN(train_data["platform_simpl"], test_data["platform_simpl"], topN=10)
train_ctry, test_ctry = cap_to_topN(train_data["conn_country"],  test_data["conn_country"],  topN=40)

ohe_cols = pd.get_dummies(
    pd.DataFrame({
        "platform_simpl": train_plat,
        "conn_country": train_ctry
    }),
    dummy_na=False
).astype("int8")

ohe_cols_test = pd.get_dummies(
    pd.DataFrame({
        "platform_simpl": test_plat,
        "conn_country": test_ctry
    }),
    dummy_na=False
).astype("int8")

# Alinear columnas
ohe_cols_test = ohe_cols_test.reindex(columns=ohe_cols.columns, fill_value=0).astype("int8")


In [9]:
# Crear user_order y chequear columnas requeridas
# --------------------------------
# - user_order: orden secuencial por usuario (1,2,3,...) según ts

# 1) user_order por usuario (en train y test)
for df in (train_data, test_data):
    # aseguramos datetime
    df["ts"] = pd.to_datetime(df["ts"], utc=True, errors="coerce")
    # orden por usuario y tiempo
    df.sort_values(["username", "ts"], inplace=True)
    df["user_order"] = df.groupby("username", observed=True).cumcount() + 1
    # volvemos a ordenar por obs_id para no alterar tu flujo
    df.sort_values("obs_id", inplace=True)

# 2) chequear que existan todas las columnas que vamos a usar
base_numeric = [
    "hour","dayofweek","is_weekend",
    "gap_offline_minutes","had_offline",
    "shuffle","offline","incognito_mode","user_order"
]

freq_cols = [f"fe_{c}" for c in big_cols]

def ensure_cols(df, cols, fill_value=0):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        # creamos las faltantes con fill_value
        for c in missing:
            df[c] = fill_value
        print("columnas creadas por faltar en df:", missing)

# aseguramos que estén todas en train y test
ensure_cols(train_data, base_numeric + freq_cols, fill_value=0)
ensure_cols(test_data,  base_numeric + freq_cols, fill_value=0)

# (opcional) tipado liviano
for df in (train_data, test_data):
    for c in ["hour","dayofweek","is_weekend","had_offline","shuffle","offline","incognito_mode","user_order"]:
        if c in df:
            if c in ["hour"]: df[c] = df[c].astype("int16")
            elif c in ["dayofweek","is_weekend","had_offline","shuffle","offline","incognito_mode"]: df[c] = df[c].astype("int8")
            elif c == "user_order": df[c] = df[c].astype("int32")
    for c in freq_cols:
        if c in df:
            df[c] = df[c].astype("int32")


In [10]:
#Construimos matriz final de features
# --------------------------------
# Combina:
# - Variables numéricas derivadas (base_numeric)
# - Columnas con Frequency Encoding (freq_cols)
# - Columnas One-Hot (ohe_cols)

base_numeric = [
    "hour","dayofweek","is_weekend",
    "gap_offline_minutes","had_offline",
    "shuffle","offline","incognito_mode","user_order"
]

freq_cols = [f"fe_{c}" for c in big_cols]

# Entrenamiento
X_train_full = pd.concat(
    [
        train_data[base_numeric + freq_cols].astype("float32").reset_index(drop=True),
        ohe_cols.reset_index(drop=True)
    ],
    axis=1
)

# Test → reindexar columnas para que coincidan con train
X_test_full = pd.concat(
    [
        test_data[base_numeric + freq_cols].astype("float32").reset_index(drop=True),
        ohe_cols_test.reset_index(drop=True)
    ],
    axis=1
).reindex(columns=X_train_full.columns, fill_value=0)

# Target
y = train_data["y"].astype("int8").to_numpy()
feature_names = X_train_full.columns

print("X_train_full:", X_train_full.shape)
print("X_test_full:", X_test_full.shape)


X_train_full: (911344, 67)
X_test_full: (51570, 67)


In [11]:
# Split temporal (validación realista)
# --------------------------------
# - Cortamos el train por tiempo: último 20% para validación.
# - Evita leakage

# Ordenar por ts y cortar por cuantiles
cut = train_data["ts"].quantile(0.80)
tr_idx = train_data["ts"] < cut
va_idx = train_data["ts"] >= cut

X_tr = X_train_full.loc[tr_idx].reset_index(drop=True)
y_tr = y[tr_idx.values]
X_va = X_train_full.loc[va_idx].reset_index(drop=True)
y_va = y[va_idx.values]

print("Train:", X_tr.shape, "Val:", X_va.shape)


Train: (729074, 67) Val: (182270, 67)


In [12]:

#Manejo de desbalance (scale_pos_weight)
# --------------------------------
# - Calculamos neg/pos para ajustar el peso de la clase positiva.
# - Ayuda a mejorar ROC-AUC cuando y=1 es minoritaria.

pos = y_tr.sum()
neg = y_tr.shape[0] - pos
scale_pos_weight = max(1.0, neg / max(1, pos))

print(f"En el train hay {pos} positivos (skip) y {neg} negativos (no skip).")
print(f"Eso significa que hay {neg/pos:.2f} veces más no-skip que skip.")
print(f"scale_pos_weight se ajusta a: {scale_pos_weight:.2f}")


En el train hay 149180 positivos (skip) y 579894 negativos (no skip).
Eso significa que hay 3.89 veces más no-skip que skip.
scale_pos_weight se ajusta a: 3.89


In [13]:
#XGBoost con GPU + early stopping
# --------------------------------
# - Definimos hiperparámetros base (incluye scale_pos_weight por desbalance).
# - Forzamos uso de GPU (device='cuda' o gpu_hist según versión).
# - Entrenamos con early_stopping (corta cuando no mejora AUC).
# - Reporta AUC-ROC en validación temporal.

import xgboost as xgb
from sklearn.metrics import roc_auc_score

base_params = dict(
    objective="binary:logistic",
    eval_metric="auc",
    random_state=42,
    learning_rate=0.05,
    max_depth=10,
    min_child_weight=2.0,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=2.0,
    n_estimators=3000,              # grande + early stopping
    scale_pos_weight=scale_pos_weight
)

# Forzar GPU según versión
try:
    clf_xgb = xgb.XGBClassifier(device="cuda", **base_params)  # XGBoost >= 2.0
except TypeError:
    clf_xgb = xgb.XGBClassifier(                               # XGBoost < 2.0
        tree_method="gpu_hist",
        predictor="gpu_predictor",
        gpu_id=0,
        **base_params
    )

clf_xgb.fit(
    X_tr, y_tr,
    eval_set=[(X_va, y_va)],
    verbose=False,
    early_stopping_rounds=100
)

# Métricas y mejor iteración
best_iter = getattr(clf_xgb, "best_iteration", None)
best_score = getattr(clf_xgb, "best_score", None)

pred_va = clf_xgb.predict_proba(X_va)[:, 1]
auc_va = roc_auc_score(y_va, pred_va)

print(f"AUC-ROC (validación temporal): {auc_va:.5f}")
if best_iter is not None:
    print(f"Mejor iteración (early stopping): {best_iter}")
if best_score is not None:
    print(f"Mejor AUC registrado por XGB: {best_score:.5f}")





/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [18:37:48] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


AUC-ROC (validación temporal): 0.79914
Mejor iteración (early stopping): 163
Mejor AUC registrado por XGB: 0.79914


In [14]:

#Random Search de hiperparámetros
# --------------------------------
# - Define un espacio de búsqueda (param_space) con valores posibles.
# - Usa ParameterSampler para probar 30 combinaciones al azar.
# - Entrena un XGBoost con cada combinación (GPU + early stopping).
# - Calcula el AUC en validación temporal.
# - Se queda con la mejor combinación encontrada (best_params).

from sklearn.model_selection import ParameterSampler
import numpy as np

param_space = {
    "max_depth": [6,8,10,12],
    "min_child_weight": [1,2,4,8],
    "subsample": [0.7,0.8,0.9],
    "colsample_bytree": [0.7,0.8,0.9],
    "reg_lambda": [1.0,2.0,3.0,5.0],
    "learning_rate": [0.03, 0.05, 0.07],
}
best_auc = -1
best_params = None
for g in ParameterSampler(param_space, n_iter=30, random_state=42):
    params = base_params.copy()
    params.update(g)
    try:
        model = xgb.XGBClassifier(device="cuda", **params)
    except TypeError:
        model = xgb.XGBClassifier(tree_method="gpu_hist", predictor="gpu_predictor", gpu_id=0, **params)
    model.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=False, early_stopping_rounds=100)
    auc = roc_auc_score(y_va, model.predict_proba(X_va)[:,1])
    if auc > best_auc:
        best_auc = auc
        best_params = params
        print(f"nuevo mejor AUC {auc:.5f} con {g}")

print("BEST AUC:", best_auc)
print("BEST PARAMS:", {k:best_params[k] for k in param_space})


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


nuevo mejor AUC 0.79359 con {'subsample': 0.9, 'reg_lambda': 5.0, 'min_child_weight': 2, 'max_depth': 6, 'learning_rate': 0.03, 'colsample_bytree': 0.8}


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


nuevo mejor AUC 0.80582 con {'subsample': 0.8, 'reg_lambda': 1.0, 'min_child_weight': 1, 'max_depth': 8, 'learning_rate': 0.03, 'colsample_bytree': 0.9}


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

nuevo mejor AUC 0.81047 con {'subsample': 0.9, 'reg_lambda': 2.0, 'min_child_weight': 4, 'max_depth': 6, 'learning_rate': 0.07, 'colsample_bytree': 0.7}


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`se

nuevo mejor AUC 0.81149 con {'subsample': 0.9, 'reg_lambda': 5.0, 'min_child_weight': 8, 'max_depth': 8, 'learning_rate': 0.07, 'colsample_bytree': 0.7}


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


BEST AUC: 0.8114873132209013
BEST PARAMS: {'max_depth': 8, 'min_child_weight': 8, 'subsample': 0.9, 'colsample_bytree': 0.7, 'reg_lambda': 5.0, 'learning_rate': 0.07}


In [15]:
final_params = best_params if 'best_params' in locals() and best_params is not None else base_params
try:
    final_model = xgb.XGBClassifier(device="cuda", **final_params)
except TypeError:
    final_model = xgb.XGBClassifier(tree_method="gpu_hist", predictor="gpu_predictor", gpu_id=0, **final_params)

final_model.fit(
    X_train_full, y,
    eval_set=[(X_va, y_va)],  # o [(X_train_full, y)] si querés entrenar con todo
    verbose=False,
    early_stopping_rounds=100
)

test_proba = final_model.predict_proba(X_test_full)[:,1]
sub = pd.DataFrame({"obs_id": test_data["obs_id"], "pred_proba": test_proba})
sub.to_csv("submission.csv", index=False)
print("submission.csv listo")


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


submission.csv listo
